In [ ]:
import os
import sys
import matplotlib.pyplot as plt

import random
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sn
import ray#, psutil

import tellurium as te

In [ ]:
with open("egfrActivation.sb") as f:
    egfrActivationModel = f.read()

model = te.loada(egfrActivationModel)
species = model.getFloatingSpeciesIds()
#model.reset()

In [ ]:
egf = np.array([0.2, 0.4, 1.0, 2.5, 5.0, 10.0, 20.0, 100.0])
pegfr = np.array([[1.538, 1.639, 2.355, 1.899, 1.935, 1.790, 1.700, 0.781, 3.072],
[2.244, 1.943, 1.395, 1.749, 2.028, 2.039, 2.087, 1.108, 2.858],
[1.501, 2.935, 4.037, 5.768, 5.986, 7.527, 6.889, 7.812, 7.490],
[2.899, 9.543, 13.878, 18.940, 23.153, 27.694, 28.414, 34.901, 37.434],
[2.504, 13.263, 21.242, 31.053, 34.163, 37.691, 36.178, 47.017, 46.832],
[2.642, 12.110, 15.855, 28.133, 30.337, 29.640, 32.854, 31.818, 44.066],
[2.069, 18.130, 13.089, 24.388, 29.611, 36.226, 32.383, 37.782, 38.338],
[1.568, 20.861, 38.950, 38.886, 32.507, 32.372, 31.802, 30.657, 38.321]])

In [ ]:
t0 = 0
tf = 80
tnum = 9
egfLevels = len(egf)

def getResidual(model, pars):
    model = setParameter(model, pars)
    residual = 1
    for i in range(egfLevels):
        model.reset()
        model.E = egf[i]
        eq = model.simulate(t0, tf, tnum)
        tempSc = np.sqrt(np.sum(((eq["[Rp]"] - pegfr[i])/(0.1 * pegfr[i]))**2) / 9)
        residual *= tempSc
    residual **= 1/egfLevels
    
    return residual

#def getObs(model, pars):
    

def setParameter(model, pars):
    model.Vr = pars[0]
    model.kf1 = pars[1]
    model.kf2 = pars[2]
    model.ki = pars[3]
    model.ke = pars[4]
    model.kp = pars[5]
    model.kd = pars[6]
    model.K1 = pars[7]
    model.K2 = pars[8]
    model.g1 = pars[9]
    model.g2 = pars[10]
    model.g3 = pars[11]
    model.g4 = pars[12]
    model.g5 = pars[13]
    model.g6 = pars[14]
    model.g7 = pars[15]
    return model

In [ ]:
def initial():
    k1 = random.gauss(np.log10(0.0016), 0.001)
    #k_1 = random.gauss(np.log10(0.004), 0.001)
    k_1 = random.uniform(-4, 2)
    kt = random.gauss(np.log10(0.0012), 0.001)
    ke = random.gauss(np.log10(0.0033), 0.001)
    Vr = random.gauss(np.log10(0.0012), 0.001)
    kx = random.gauss(np.log10(1), 0.001)
    kh = random.gauss(np.log10(0.0004), 0.001)
    k2 = random.gauss(np.log10(0.082), 0.001)
    k_2 = random.uniform(-4, 2)
    k3 = random.gauss(np.log10(1.2), 0.001)
    k_3 = random.uniform(-4, 2)
    return np.array([k1, k_1, kt, ke, Vr, kx, kh, k2, k_2, k3, k_3])

In [ ]:
def prior(pars):
    k1p = sp.stats.norm(np.log10(0.0016), 0.1).pdf(pars[0])
    #k_1p = sp.stats.norm(np.log10(0.004), 0.001).pdf(pars[1])
    k_1p = 1/6
    if pars[1] < -4 or pars[1] > 2:
        k_1p = 0
    ktp = sp.stats.norm(np.log10(0.0012), 0.1).pdf(pars[2])
    kep = sp.stats.norm(np.log10(0.0033), 0.1).pdf(pars[3])
    Vrp = sp.stats.norm(np.log10(0.0012), 0.1).pdf(pars[4])
    kxp = sp.stats.norm(np.log10(1.0), 0.1).pdf(pars[5])
    khp = sp.stats.norm(np.log10(0.0004), 0.1).pdf(pars[6])
    k2p = sp.stats.norm(np.log10(0.082), 0.1).pdf(pars[7])
    k_2p = 1/6
    if pars[8] < -4 or pars[8] > 2:
        k_2p = 0
    k3p = sp.stats.norm(np.log10(1.2), 0.1).pdf(pars[9])
    k_3p = 1/6
    if pars[10] < -4 or pars[10] > 2:
        k_3p = 0
        
    return np.prod([k1p, k_1p, ktp, kep, Vrp, kxp, khp, k2p, k_2p, k3p, k_3p])

In [ ]:
def getNewPars(pars):
    newPars = pars + np.random.normal(0,0.01,len(pars))
    return(newPars)

In [ ]:
#@ray.remote
def mcmc(model, steps, burnin, index=0):
    curPars = initial()
    priorP = prior(curPars)
    curPrior = priorP
    curFit = getResidual(model, 10 ** curPars)
    burninset = []
    burninscore = []
    gen = 0
    while gen < burnin:
        newPars = getNewPars(curPars)
        priorP = prior(newPars)
        if priorP > 0 :
            newFit = getResidual(model, 10 ** newPars)
            thr = np.exp((curFit - newFit)) * priorP / curPrior
            if thr > 1 or np.random.random() < thr :
                curPars = newPars
                curPrior = priorP
                curFit = newFit
                burninset.append(curPars)
                burninscore.append(curFit)
                gen += 1
    np.savetxt('burnin-set-' + str(index) + '.txt',np.array(burninset))
    np.savetxt('burnin-score-' + str(index) + '.txt',np.array(burninscore))
    mcmcset = []
    mcmcscore = []
    gen = 0
    while gen < steps:
        newPars = getNewPars(curPars)
        priorP = prior(newPars)
        if priorP > 0 :
            newFit = getResidual(model, 10 ** newPars)
            thr = np.exp((curFit - newFit)) * priorP / curPrior
            if thr > 1 or np.random.random() < thr :
                curPars = newPars
                curPrior = priorP
                curFit = newFit
                mcmcset.append(curPars)
                mcmcscore.append(curFit)
                gen += 1
    np.savetxt('mcmc-set-' + str(index) + '.txt',np.array(mcmcset))
    np.savetxt('mcmc-score-' + str(index) + '.txt',np.array(mcmcscore))

In [ ]:
steps = 1000000
burnin = 50000
#cpuNum = os.cpu_count()
#ray.init(num_cpus=cpuNum)
#ray.get([mcmc.remote(model,steps,burnin,i) for i in range(cpuNum)])
mcmc(model,steps,burnin,0)